In [1]:
import time
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm 

In [2]:
data = pd.read_csv("../paruvendu_api/outputs/paruvendu.csv")
data.head()

,location,version,price,year,mileage,fuel_type,emission,transmission,door_nb,technical_power,seat_nb,actual_power,body_type
0,86.0,dacia/duster,17600.0,2019,18300.0,essence,NaN,manuelle,4.0,7.0,5.0,130.0,4x4
1,59.0,mini/cooper,24990.0,2018,47500.0,essence,119.0,automatique,2.0,10.0,4.0,192.0,berline
2,53.0,nissan/qashqai,8490.0,2013,166400.0,diesel,129.0,manuelle,4.0,6.0,5.0,110.0,break
3,1.0,nissan/qashqai,16900.0,2016,61500.0,diesel,128.0,automatique,4.0,7.0,5.0,130.0,4x4
4,30.0,mini/cooper,17900.0,2018,58529.0,diesel,99.0,manuelle,4.0,5.0,5.0,95.0,berline


In [3]:
df = data.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15552 entries, 0 to 15551
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location         15534 non-null  float64
 1   version          15552 non-null  object 
 2   price            15542 non-null  float64
 3   year             15552 non-null  int64  
 4   mileage          15544 non-null  float64
 5   fuel_type        14492 non-null  object 
 6   emission         9052 non-null   float64
 7   transmission     11061 non-null  object 
 8   door_nb          15102 non-null  float64
 9   technical_power  14779 non-null  float64
 10  seat_nb          10239 non-null  float64
 11  actual_power     7602 non-null   float64
 12  body_type        13197 non-null  object 
dtypes: float64(8), int64(1), object(4)
memory usage: 1.5+ MB


In [5]:
df.isna().sum()

location             18
version               0
price                10
year                  0
mileage               8
fuel_type          1060
emission           6500
transmission       4491
door_nb             450
technical_power     773
seat_nb            5313
actual_power       7950
body_type          2355
dtype: int64

## TRAINTEST - NETTOYAGE - ENCODAGE

### Nettoyage

In [6]:
df_agg = df.groupby('version', as_index=False).median()
df_agg.head()

,version,location,price,year,mileage,emission,door_nb,technical_power,seat_nb,actual_power
0,abarth/500,58.0,19999.0,2019.0,12932.0,155.0,2.0,8.0,4.0,165.0
1,abarth/595,31.0,20490.0,2017.0,32600.0,139.0,2.0,10.0,4.0,180.0
2,audi/a1,37.5,17490.0,2017.0,40500.0,103.0,4.0,5.0,4.0,95.0
3,audi/a3,51.0,19490.0,2016.0,78451.0,113.0,4.0,6.0,5.0,125.0
4,audi/a4,38.0,24999.0,2017.0,69651.0,139.0,4.0,8.0,5.0,150.0


In [7]:
""" for name in tqdm(df_agg.version):
    for col in df.select_dtypes('float64'):
        value = df_agg[df_agg['version'] == name][col].item()
        df.loc(df['version'].where(name)[col].fillna(value, inplace=True))
        df[col] = df[col].fillna(value) """

" for name in tqdm(df_agg.version):\n    for col in df.select_dtypes('float64'):\n        value = df_agg[df_agg['version'] == name][col].item()\n        df.loc(df['version'].where(name)[col].fillna(value, inplace=True))\n        df[col] = df[col].fillna(value) "

In [8]:
df.head()

,location,version,price,year,mileage,fuel_type,emission,transmission,door_nb,technical_power,seat_nb,actual_power,body_type
0,86.0,dacia/duster,17600.0,2019,18300.0,essence,NaN,manuelle,4.0,7.0,5.0,130.0,4x4
1,59.0,mini/cooper,24990.0,2018,47500.0,essence,119.0,automatique,2.0,10.0,4.0,192.0,berline
2,53.0,nissan/qashqai,8490.0,2013,166400.0,diesel,129.0,manuelle,4.0,6.0,5.0,110.0,break
3,1.0,nissan/qashqai,16900.0,2016,61500.0,diesel,128.0,automatique,4.0,7.0,5.0,130.0,4x4
4,30.0,mini/cooper,17900.0,2018,58529.0,diesel,99.0,manuelle,4.0,5.0,5.0,95.0,berline


In [9]:
# RESTART TON KERNEL BLÉRO

loop_length = len(df)
df.fillna(-1.0, inplace=True)

for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    line_agg = df_agg.loc[df_agg['version'] == line['version']]
    if line['emission'] == -1.0:
        n_emission = line_agg['emission'].item()
        line['emission'] = n_emission
    if line['seat_nb'] == -1.0:
        n_seat_nb = line_agg['seat_nb'].item()
        line['seat_nb'] = n_seat_nb
    if line['door_nb'] == -1.0:
        n_door_nb = line_agg['door_nb'].item()
        line['door_nb'] = n_door_nb
    if line['actual_power'] == -1.0:
        n_actual_power = line_agg['actual_power'].item()
        line['actual_power'] = n_actual_power
    if line['technical_power'] == -1.0:
        n_technical_power = line_agg['technical_power'].item()
        line['technical_power'] = n_technical_power
    df.iloc[i] = line
df.replace(-1.0, None, inplace=True)

100%|██████████| 15552/15552 [00:10<00:00, 1493.31it/s]


In [10]:
(data.isna().sum()/data.shape[0]).sort_values(ascending=False)

actual_power       0.511188
emission           0.417953
seat_nb            0.341628
transmission       0.288773
body_type          0.151427
fuel_type          0.068158
technical_power    0.049704
door_nb            0.028935
location           0.001157
price              0.000643
mileage            0.000514
version            0.000000
year               0.000000
dtype: float64

In [31]:
transmission_list = df['transmission'].unique()
transmission_list = transmission_list[0:2]
body_type_list_raw = data['body_type'].unique()
model_list = df['version'].unique()

dict_model_transmission = dict()
for model in model_list:
    dict_model_transmission[model] = dict()
    for transmission in transmission_list:
        dict_model_transmission[model][transmission] = 0

loop_length = len(data)
for i in range(loop_length):
    line = data.iloc[i].copy()
    model = line['version']
    transmission = line['transmission']
    if transmission != -1:
        dict_model_transmission[model][transmission] += 1

for key in dict_model_transmission.keys():
    dict_model_transmission[key] = max(dict_model_transmission[key],key=dict_model_transmission[key].get)

body_type_list = []
for i in range(len(body_type_list_raw)):
    value = body_type_list_raw[i]
    if value != -1:
        body_type_list.append(value)
print(body_type_list)

dict_model_body_type = dict()
for model in model_list:
    dict_model_body_type[model] = dict()
    for body_type in body_type_list:
        dict_model_body_type[model][body_type] = 0

loop_length = len(data)
for i in range(loop_length):
    line = data.iloc[i].copy()
    model = line['version']
    body_type = line['body_type']
    if body_type != -1:
        dict_model_body_type[model][body_type] += 1

for key in dict_model_body_type.keys():
    dict_model_body_type[key] = max(dict_model_body_type[key],key=dict_model_body_type[key].get)

fuel_type_list_raw = data['fuel_type'].unique()
fuel_type_list = fuel_type_list_raw[0:2]

dict_model_fuel_type = dict()
for model in model_list:
    dict_model_fuel_type[model] = dict()
    for fuel_type in fuel_type_list:
        dict_model_fuel_type[model][fuel_type] = 0

loop_length = len(data)
for i in range(loop_length):
    line = data.iloc[i].copy()
    model = line['version']
    fuel_type = line['fuel_type']
    if fuel_type != -1:
        dict_model_fuel_type[model][fuel_type] += 1

for key in dict_model_fuel_type.keys():
    dict_model_fuel_type[key] = max(dict_model_fuel_type[key],key=dict_model_fuel_type[key].get)

loop_length = len(data)
for i in range(loop_length):
    line = data.iloc[i].copy()
    model = line['version']
    if line['transmission'] == -1:
        n_transmission = dict_model_transmission[model]
        line['transmission'] = n_transmission
    if line['body_type'] == -1:
        n_body_type = dict_model_body_type[model]
        line['body_type'] = n_body_type
    if line['fuel_type'] == -1:
        n_fuel_type = dict_model_fuel_type[model]
        line['fuel_type'] = n_fuel_type
    data.iloc[i] = line
data.replace(-1,None,inplace=True)

KeyError: nan

In [ ]:
data.info()

,location,version,price,year,mileage,fuel_type,emission,transmission,door_nb,technical_power,seat_nb,actual_power,body_type
0,86.0,dacia/duster,17600.0,2019,18300.0,essence,126.0,manuelle,4.0,7.0,5.0,130.0,4x4
1,59.0,mini/cooper,24990.0,2018,47500.0,essence,119.0,automatique,2.0,10.0,4.0,192.0,berline
2,53.0,nissan/qashqai,8490.0,2013,166400.0,diesel,129.0,manuelle,4.0,6.0,5.0,110.0,break
3,1.0,nissan/qashqai,16900.0,2016,61500.0,diesel,128.0,automatique,4.0,7.0,5.0,130.0,4x4
4,30.0,mini/cooper,17900.0,2018,58529.0,diesel,99.0,manuelle,4.0,5.0,5.0,95.0,berline


In [ ]:
(data.isna().sum()/data.shape[0]).sort_values(ascending=False)
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
df.isna().sum()

location             18
version               0
price                10
year                  0
mileage               8
fuel_type          1060
emission             44
transmission       4491
door_nb             100
technical_power       0
seat_nb              60
actual_power        153
body_type          2355
dtype: int64

In [ ]:
data.to_csv('../paruvendu_api/outputs/clean_dataset.csv',index=None)

In [ ]:
a

### Encodage

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def encodage_feature(df, column):
    ohe = OneHotEncoder(sparse=False)
    feature_array = ohe.fit_transform(df[[column]])
    feature_labels = ohe.categories_
    feature_labels = np.array(feature_labels).ravel()
    features = pd.DataFrame(feature_array, columns=feature_labels)
    
    return features

In [ ]:
def preprocessing(df):    
    for col in df.select_dtypes('object'):
        features = encodage_feature(df, col)
        df = pd.concat([df, features], axis=1)
        df = df.drop(columns=[col])
    
    X = df.drop('price', axis=1)
    y = df['price']
    y = y.values.reshape(y.shape[0], 1)

    print(X.shape, y.shape)
    
    return X, y

In [ ]:
X, y = preprocessing(df)

In [ ]:
x.head()

In [ ]:
y.head()

### Traintest

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

### Standardiser

La phase exploratoire nous a permis d'observer une distribution normale, plus appropriée pour standardiser les données

In [ ]:
from sklearn import preprocessing

stds = preprocessing.StandardScaler()
X1_train = stds.fit_transform(X_train)

In [ ]:
X1_train

## MODELISATION
### 1.1 Phase d'entrainement avec Scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# starting time
start = time.time()

#entrainement du modèle
lmodellineaire_std = LinearRegression()
lmodellineaire_std.fit(X1_train, y_train)

# sleeping for 1 sec to get 10 sec runtime
time.sleep(1)

# program body ends

# end time
end = time.time()
time_fit_std = end - start

# total time taken
print(f"Runtime of the program is {time_fit_std}")

In [ ]:
print('Intercept: \n', lmodellineaire_std.intercept_)
print('Coefficients: \n', lmodellineaire_std.coef_)

In [ ]:
# Evaluation du training set
y_train_predict = lmodellineaire_std.predict(X1_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
r2 = r2_score(y_train, y_train_predict)

print("La performance du modèle sur la base dapprentissage")
print("--------------------------------------")
print(f"L'erreur quadratique moyenne est {rmse}")
print(f"Le score R2 est {r2}")

In [ ]:
X1_train

In [ ]:
plt.scatter(X1_train[:, 3], y_train) # observer le kilométrage
plt.scatter(X1_train[:, 3], y_train_predict, c='r')

In [ ]:
X1_test = stds.transform(X_test)

In [ ]:
# model evaluation for testing set
y_test_predict = lmodellineaire_std.predict(X1_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
r2 = r2_score(y_test, y_test_predict)

print("La performance du modèle sur la base de test")
print("--------------------------------------")
print(f"L'erreur quadratique moyenne est {rmse}")
print(f"Le score R2 est {r2}")

In [ ]:
plt.scatter(X1_test[:, 3], y_test)
plt.scatter(X1_test[:, 3], y_test_predict, c='r')

## FEATURE SELECTION

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
selector = VarianceThreshold(0.6)
selector.fit_transform(X_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
selector = SelectKBest(f_regression, k=2)
selector.fit_transform(X_train, y_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
a

In [ ]:
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(X_train, y_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()